In [ ]:
# 1. Importy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import albumentations as A

Preprocessing i wczytanie danych

In [ ]:
# 2. Wczytanie i podział danych
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
images = np.concatenate([train_images, test_images], axis=0)
labels = np.concatenate([train_labels, test_labels], axis=0)
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=10, stratify=labels
)

# 3. Preprocessing: normalizacja i reshape
X_train = X_train.astype('float32') / 255.0         # Podział pikseli na coś pomiędzy 0-1
X_test = X_test.astype('float32') / 255.0
X_train = X_train[..., np.newaxis]  # Kształt: [N, 28, 28, 1]
X_test = X_test[..., np.newaxis]

Budowa modelu NN

In [ ]:
# 4. Budowa modelu CNN
def build_model():
    inp = layers.Input(shape=(28, 28, 1))                                   # Rozmiar obrazka 28 x 28 pikseli i 1 kanal koloru
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005))(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()

Augmentacja z Keras i Albumentations

In [ ]:
 # 5. Augmentacja: połączenie Keras i Albumentations
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size, keras_gen, alb_pipe):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.keras_gen = keras_gen
        self.alb_pipe = alb_pipe
        self.indices = np.arange(len(X))
        np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.X))
        batch_indices = self.indices[start_idx:end_idx]
        batch_X = self.X[batch_indices].copy()
        batch_y = self.y[batch_indices]

        # Zastosuj augmentację Keras dla każdego obrazu
        for i in range(len(batch_X)):
            batch_X[i] = self.keras_gen.random_transform(batch_X[i])
            # Albumentations: wymaga formatu [28, 28]
            aug = self.alb_pipe(image=batch_X[i].squeeze())['image']
            batch_X[i] = aug[..., np.newaxis]

        return batch_X, batch_y

# Definicja augmentacji
keras_gen = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    shear_range=0.05
)

alb_pipe = A.Compose([
    A.Rotate(limit=8, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=8, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    A.GaussNoise(var_limit=(10.0, 30.0), p=0.3)
])

# 6. Trening z augmentacją i optymalizacją
batch_size = 64
train_gen = CustomDataGenerator(X_train, y_train, batch_size, keras_gen, alb_pipe)

early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(
    train_gen,
    steps_per_epoch=int(np.ceil(len(X_train) / batch_size)),  # Poprawka na TypeError
    validation_data=(X_test, y_test),
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)

Ewaluacja i wizualizacja

In [ ]:
# 7. Ewaluacja modelu
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test Accuracy: {acc:.4f}")

# 8. Zapis modelu
model.save('fashion_mnist_cnn_augmented.keras')

# 9. Interfejs predykcji
from tensorflow.keras.models import load_model
saved_model = load_model('fashion_mnist_cnn_augmented.keras')

labels_map = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

def predict_and_show(img):
    if img.max() > 1:
        img = img.astype('float32') / 255.0
    inp = img[np.newaxis, ..., np.newaxis]
    pred = saved_model.predict(inp)
    idx = np.argmax(pred)
    plt.figure(figsize=(3, 3))
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(f"Predicted: {idx} - {labels_map[idx]}")
    plt.axis('off')
    plt.show()

# Przykład użycia
for i in range(5):
    predict_and_show(X_test[i])